In [ ]:
!pip install datasets
!pip install transformers[torch]

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

import torch
import pandas as pd
import json


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

text = "Replace me by any text you'd like.", "Hello"
encoded_input = tokenizer(text, return_tensors='pt', padding=True, padding_side='left')

# output = model(**encoded_input)

input_ids = encoded_input['input_ids']

# Generate text response
max_length = 50  # Maximum length of the generated response
temperature = 0.7  # Temperature parameter for sampling
top_k = 50  # Top-k sampling parameter
top_p = 0.9  # Top-p sampling parameter

# Generate text using the model
output = model.generate(
    input_ids,
    max_length=max_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    num_return_sequences=1
)

# Decode the generated token IDs into text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated text:", generated_text)

Keyword arguments {'padding_side': 'left'} not recognized.
Keyword arguments {'padding_side': 'left'} not recognized.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated text: Replace me by any text you'd like. I can't promise that I'll get a reply, but I'm pretty sure I'll be able to. I'm not going to ask you to do anything for me, and I'll be fine


In [ ]:
from datasets import load_dataset, DatasetDict, load_from_disk, Dataset

In [ ]:
# Create prompt tensors from prompts_contexts.csv file.

# Read the CSV file into a pandas DataFrame
prompts_contexts = '../preprocessing/updated_prompts_contexts.csv'
# prompts_contexts = '/content/drive/MyDrive/CSCI499 Natural Language Processing/preprocessing/updated_prompts_contexts.csv'
df = pd.read_csv(prompts_contexts)
dataset = Dataset.from_pandas(df)

def create_mult_seq(example):
    sentences = []
    # Concatenate sentences from all fields
    for column, value in example.items():
        sentences.extend(value.split('.'))  # Split sentences based on '. ' delimiter

    example['Sequences'] = sentences
    tokens = tokenizer(example['Sequences'])
    examples['input_ids'] = tokens['input_ids']
    # examples['mask???'] = tokens['mask???']
    return example

def tokenize(example):
  example = tokenizer(example['Prompt'], example['Context'], example['Updated Context'], truncation=True, padding=True, return_tensors="pt")
  return example

tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset = tokenized_dataset.remove_columns(['Prompt', 'Context', 'Updated Context', 'labels'])
print(tokenized_dataset)

# Split the dataset into train/test/valid (80/10/10)
train_test_ds = tokenized_dataset.train_test_split(test_size=.2)
test_valid_ds = train_test_ds['test'].train_test_split(test_size=.5)

ds = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_ds['test'],
    'valid': test_valid_ds['train']})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})


In [ ]:
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

train_dataloader = DataLoader(
    ds["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    ds["valid"], batch_size=8, collate_fn=data_collator
)


# Generate text response
max_length = 600  # Maximum length of the generated response
temperature = 0.7  # Temperature parameter for sampling
top_k = 50  # Top-k sampling parameter
top_p = 0.9  # Top-p sampling parameter

num_epochs = 1
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v for k, v in batch.items()}

        # Generate text using the model
        output = model.generate(
            # input_ids,
            **batch,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            num_return_sequences=1
        )

        # Decode the generated token IDs into text
        generated_text = tokenizer.batch_decode(output, skip_special_tokens=True)

        [print("Generated text:", text) for text in generated_text]
    break

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated text: Given the following set of drugs, decide if the synergy of the drug combination is synergistic or antagonistic: Gemcitabine, PictilisibPictilisib and Gemcitabine have a Loewe score of: -0.1477, HSA score of: -0.5063, and ZIP score of: -0.6687.The results of this study indicate that the synergistic effect of the drug combination is achieved only when the combination of Gemcitabine, Pictilisib and Pictilisib is combined with a drug that has a Loewe score of: -0.1477, HSA score of: -0.5063, and ZIP score of: -0
Generated text: Given the following set of drugs, decide if the synergy of the drug combination is synergistic or antagonistic: Trametinib, Alvespimycin, Dasatinib, Pemetrexed, GSK2334470Alvespimycin and GSK2334470 have a Loewe score of: -0.3629, HSA score of: -0.3789, and ZIP score of: -0.7793. GSK2334470 and Pemetrexed have a Loewe score of: -0.1537, HSA score of: -0.2081, and ZIP score of: -0.8464. Dasatinib and GSK2334470 have a Loewe score of: -0.379, HSA score